In [35]:
import pandas as pd
import numpy as np
import datetime
import altair as alt
import os

Let's read the data:

In [36]:
DATA_DIR = os.environ['DATA_DIR']

In [37]:
narcotics = pd.read_csv(DATA_DIR + "/raw/edited_clean_narcotics_facilities.csv") 

In [38]:
mental_health = pd.read_csv(DATA_DIR + "/raw/edited_from_full_to_mental_health_facilities.csv") 

In [39]:
psych = pd.read_csv(DATA_DIR + "/raw/edited_mental_health_psych_cleaned_geo.csv")

In [40]:
substance_abuse = pd.read_csv(DATA_DIR + "/raw/edited_substance_use_cleaned.csv") 

Let's put the data together:

In [41]:
frames = civilian_officer_frames = [narcotics, mental_health, substance_abuse, psych]

In [42]:
all_frames = pd.concat(frames)

In [43]:
all_frames

,facility_name,facility_type,address,city,county,state,zip,capacity,latitude,longitude,phone_number,other_details
0,"California Forensic Medical Group, Inc",narcotics,5325 Broder Boulevard,Dublin,Alameda County,CA,94568,120,37.715926,-121.884151,(925) 551-6700,NaN
1,Humanistic Alternatives to Addiction Research ...,narcotics,20094 Mission Boulevard,Hayward,Alameda County,CA,94541,400,37.687185,-122.102244,(510) 727-9755,NaN
2,"Addiction Research and Treatment, Inc.",narcotics,1124 International Boulevard,Oakland,Alameda County,CA,94606,650,37.791022,-122.248455,(510) 533-0800,NaN
3,"BAART Behavioral Health Services, Inc.",narcotics,1124 International Boulevard,Oakland,Alameda County,CA,94606,50,37.791022,-122.248455,(510) 533-0800,NaN
4,Successful Alternatives For Addiction & Counse...,narcotics,795 Fletcher Lane,Hayward,Alameda County,CA,94544,365,37.665521,-122.080495,(510) 247-8300,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Telecare Riverside County Psychiatric Health F...,PHF,47-915 Oasis Street,Indio,Riverside County,CA,92201,59,33.712015,-116.220702,(760) 863-8650,Co-Ed/Adult
60,Telecare Santa Cruz County Psychiatric Health ...,PHF,2250 Soquel Avenue,Santa Cruz,Santa Cruz County,CA,95062,16,36.983218,-121.993896,(831) 600-2801,Co-Ed/Adult
61,Telecare Stanislaus County PHF,PHF,1904 Richland Avenue,Ceres,Stanislaus County,CA,95307,16,37.600237,-120.963791,(209) 300-8800,Co-Ed/Adult
62,Telecare Willow Rock Center,PHF,2050 Fairmont Drive,San Leanro,Alameda County,CA,94578,16,37.709491,-122.118384,(510) 895-5502,Co-Ed/Children


In [44]:
# Stand. rows:

all_frames['facility_type'] = all_frames['facility_type'].str.upper()
all_frames['county'] = all_frames['county'].str.upper()
all_frames.head(3)
all_frames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2132 entries, 0 to 63
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   facility_name  2132 non-null   object 
 1   facility_type  2132 non-null   object 
 2   address        2132 non-null   object 
 3   city           2132 non-null   object 
 4   county         2132 non-null   object 
 5   state          2132 non-null   object 
 6   zip            2132 non-null   object 
 7   capacity       2132 non-null   int64  
 8   latitude       2132 non-null   float64
 9   longitude      2132 non-null   float64
 10  phone_number   2122 non-null   object 
 11  other_details  1811 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 216.5+ KB


In [45]:
# Create a new column with the full address

all_frames["full_address"] = all_frames["address"] + ',' + ' ' + all_frames["city"] + ',' + ' ' + all_frames["county"] + ',' + ' ' + all_frames["state"] + ',' + ' ' + all_frames["zip"].astype(str)
all_frames

,facility_name,facility_type,address,city,county,state,zip,capacity,latitude,longitude,phone_number,other_details,full_address
0,"California Forensic Medical Group, Inc",NARCOTICS,5325 Broder Boulevard,Dublin,ALAMEDA COUNTY,CA,94568,120,37.715926,-121.884151,(925) 551-6700,NaN,"5325 Broder Boulevard, Dublin, ALAMEDA COUNTY,..."
1,Humanistic Alternatives to Addiction Research ...,NARCOTICS,20094 Mission Boulevard,Hayward,ALAMEDA COUNTY,CA,94541,400,37.687185,-122.102244,(510) 727-9755,NaN,"20094 Mission Boulevard, Hayward, ALAMEDA COUN..."
2,"Addiction Research and Treatment, Inc.",NARCOTICS,1124 International Boulevard,Oakland,ALAMEDA COUNTY,CA,94606,650,37.791022,-122.248455,(510) 533-0800,NaN,"1124 International Boulevard, Oakland, ALAMEDA..."
3,"BAART Behavioral Health Services, Inc.",NARCOTICS,1124 International Boulevard,Oakland,ALAMEDA COUNTY,CA,94606,50,37.791022,-122.248455,(510) 533-0800,NaN,"1124 International Boulevard, Oakland, ALAMEDA..."
4,Successful Alternatives For Addiction & Counse...,NARCOTICS,795 Fletcher Lane,Hayward,ALAMEDA COUNTY,CA,94544,365,37.665521,-122.080495,(510) 247-8300,NaN,"795 Fletcher Lane, Hayward, ALAMEDA COUNTY, CA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Telecare Riverside County Psychiatric Health F...,PHF,47-915 Oasis Street,Indio,RIVERSIDE COUNTY,CA,92201,59,33.712015,-116.220702,(760) 863-8650,Co-Ed/Adult,"47-915 Oasis Street, Indio, RIVERSIDE COUNTY, ..."
60,Telecare Santa Cruz County Psychiatric Health ...,PHF,2250 Soquel Avenue,Santa Cruz,SANTA CRUZ COUNTY,CA,95062,16,36.983218,-121.993896,(831) 600-2801,Co-Ed/Adult,"2250 Soquel Avenue, Santa Cruz, SANTA CRUZ COU..."
61,Telecare Stanislaus County PHF,PHF,1904 Richland Avenue,Ceres,STANISLAUS COUNTY,CA,95307,16,37.600237,-120.963791,(209) 300-8800,Co-Ed/Adult,"1904 Richland Avenue, Ceres, STANISLAUS COUNT..."
62,Telecare Willow Rock Center,PHF,2050 Fairmont Drive,San Leanro,ALAMEDA COUNTY,CA,94578,16,37.709491,-122.118384,(510) 895-5502,Co-Ed/Children,"2050 Fairmont Drive, San Leanro, ALAMEDA COUNT..."


In [51]:
all_frames.to_csv(r'all_facilities.csv') # Create a csv with all the dataframes.